**Enable autoreload extension**

In [1]:
%load_ext autoreload
%autoreload 2

**Setup dependencies**

In [ ]:
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo
!git clone https://github.com/huawei-noah/SMARTS 2> /dev/null

In [ ]:
!cd SMARTS && ls && git status && git checkout ipynb-test-deps && git status && pip install .[camera-obs]

In [9]:
import sys
sys.path.insert(0, "./SMARTS/examples/env")
sys.path.insert(0, "./SMARTS/zoo")
import baseline
import policies.keep_lane_agent

**Import Base Modules**

In [10]:
import gym

from smarts.zoo import registry
from smarts.env.wrappers.single_agent import SingleAgent
from smarts.env.wrappers.episode_logger import EpisodeLogger
from smarts.env.wrappers.record import RecordVideo, RenderVideo

**Run an episode**

In [ ]:
from smarts.core.utils.episodes import episode_range

#FormatObs should already be applied
env = gym.make("figure_eight-v0")
env = SingleAgent(env)
# gym.wrappers.Monitor
env: gym.Env = RecordVideo(env, frequency=10)
env: gym.Env = RenderVideo(env)
env: gym.Env = EpisodeLogger(env)

agent = registry.make_agent("zoo.policies:keep_lane_agent")
for episode in episode_range(max_steps=8000):
    observation = env.reset()

    reward, done, info = None, False, None
    while episode.register_step(observation, reward, done, info):
        action = agent.act(observation)
        observation, reward, done, info = env.step(action)

env.close()